
*   Madhav Sood IMT2021009
*   Vineet Priyedarshi IMT2021018
*   Shlok Agrawal IMT2021103


In [38]:
!gdown 18Gn0ndFQooqUXzuP_aP2aEwUFZ5SKp0e
!gdown 13urXnrbu92GVmqFIkGS4mh7Mc6X0OI5-
!gdown 1FBugfSuiKYyATK60LbKtEJgCwAG5IU5Q

Downloading...
From: https://drive.google.com/uc?id=18Gn0ndFQooqUXzuP_aP2aEwUFZ5SKp0e
To: /content/train.csv
100% 12.6M/12.6M [00:00<00:00, 144MB/s]
Downloading...
From: https://drive.google.com/uc?id=13urXnrbu92GVmqFIkGS4mh7Mc6X0OI5-
To: /content/test.csv
100% 5.34M/5.34M [00:00<00:00, 193MB/s]
Downloading...
From: https://drive.google.com/uc?id=1FBugfSuiKYyATK60LbKtEJgCwAG5IU5Q
To: /content/sample_submission.csv
100% 418k/418k [00:00<00:00, 118MB/s]


In [39]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import urllib
import sklearn
from sklearn import svm
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns
import xgboost as xgb
from sklearn.ensemble import GradientBoostingClassifier
from keras.utils import to_categorical

In [40]:
test_df = pd.read_csv("/content/test.csv")
df = pd.read_csv("/content/train.csv")


print(df.shape)
print(df['patient_id'].nunique())

(71236, 50)
54151


In [41]:
train_frequency = df['patient_id'].value_counts().to_dict()
test_frequency = test_df['patient_id'].value_counts().to_dict()

# Combine train and test frequencies
combined_frequency = {key: train_frequency.get(key, 0) + test_frequency.get(key, 0) for key in set(train_frequency) | set(test_frequency)}

# Add 'frequency' column to both DataFrames
df['frequency'] = df['patient_id'].map(combined_frequency)
test_df['frequency'] = test_df['patient_id'].map(combined_frequency)


In [42]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

In [43]:
# Extract features and labels
X = df.drop(['enc_id', 'patient_id', 'readmission_id', 'repaglinide', 'nateglinide', 'chlorpropamide',
             'acetohexamide', 'tolbutamide', 'acarbose', 'miglitol', 'troglitazone',
             'tolazamide', 'examide', 'citoglipton', 'glyburide-metformin', 'glipizide-metformin',
             'metformin-rosiglitazone', 'metformin-pioglitazone', 'glimepiride-pioglitazone'], axis=1)

Y = df['readmission_id']
test_df = test_df.drop(['enc_id', 'patient_id', 'repaglinide', 'nateglinide', 'chlorpropamide',
             'acetohexamide', 'tolbutamide', 'acarbose', 'miglitol', 'troglitazone',
             'tolazamide', 'examide', 'citoglipton', 'glyburide-metformin', 'glipizide-metformin',
             'metformin-rosiglitazone', 'metformin-pioglitazone', 'glimepiride-pioglitazone'], axis=1)


numerical_columns = X.select_dtypes(include=['int64', 'float64']).columns
categorical_columns = X.select_dtypes(include=['object']).columns

In [44]:
# Impute missing values for numerical columns
numerical_imputer = SimpleImputer(strategy='most_frequent')
X[numerical_columns] = numerical_imputer.fit_transform(X[numerical_columns])
test_df[numerical_columns] = numerical_imputer.transform(test_df[numerical_columns])

# Impute missing values for categorical columns
categorical_imputer = SimpleImputer(strategy='most_frequent')
X[categorical_columns] = categorical_imputer.fit_transform(X[categorical_columns])
test_df[categorical_columns] = categorical_imputer.transform(test_df[categorical_columns])

# One-hot encode categorical columns
categorical_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
X_encoded = categorical_encoder.fit_transform(X[categorical_columns])
test_data_processed = categorical_encoder.transform(test_df[categorical_columns])

# Combine numerical and encoded categorical features
X_final = pd.concat([X[numerical_columns], pd.DataFrame(X_encoded, columns=categorical_encoder.get_feature_names_out(categorical_columns))], axis=1)
test_df_encoded = pd.concat([test_df[numerical_columns], pd.DataFrame(test_data_processed, columns=categorical_encoder.get_feature_names_out(categorical_columns))], axis=1)

# Split the training data for training and validation
#X_train, X_test, Y_train, Y_test = train_test_split(X_final, Y, test_size=0.2, random_state=42)


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [45]:
# Standardize the numerical columns
# scaler = StandardScaler()

# # Standardize the numerical columns in the training data
# X_final[numerical_columns] = scaler.fit_transform(X_final[numerical_columns])

# # Use the same scaler to standardize the numerical columns in the test data
# X_final[numerical_columns] = scaler.transform(X_final[numerical_columns])

# X_test_data = test_df_encoded

# # Standardize the numerical columns in the test data (assuming test_df_encoded is your test data)
# X_test_data[numerical_columns] = scaler.transform(X_test_data[numerical_columns])


# test_df_encoded = X_test_data

# print(X_train.shape)

In [46]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score
from pprint import pprint

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import optuna

import gc
import time

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [47]:
class ANN(nn.Module):
    def __init__(
        self,
        in_dim: int,
        hidden_dim_1: int,
        hidden_dim_2: int,
        hidden_dim_3: int,
        hidden_dim_4: int,
        # hidden_dim_5: int,
        n_classes:int = 3,
        dropout: float = 0.3
    ):
        super().__init__()

        self.layer1 = nn.Sequential(
            nn.Linear(in_features=in_dim, out_features=hidden_dim_1),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim_1),
            nn.Dropout(dropout),
        )
        self.layer2 = nn.Sequential(
            nn.Linear(in_features=hidden_dim_1, out_features=hidden_dim_2),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim_2),
            nn.Dropout(dropout),
        )
        self.layer3 = nn.Sequential(
            nn.Linear(in_features=hidden_dim_2, out_features=hidden_dim_3),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim_3),
            nn.Dropout(dropout),
        )
        self.layer4 = nn.Sequential(
            nn.Linear(in_features=hidden_dim_3, out_features=hidden_dim_4),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim_4),
            nn.Dropout(dropout),
        )
        # self.layer5 = nn.Sequential(
        #     nn.Linear(in_features=hidden_dim_4, out_features=hidden_dim_5),
        #     nn.ReLU(),
        #     nn.BatchNorm1d(hidden_dim_5),
        #     nn.Dropout(dropout),
        # )
        self.output_layer = nn.Linear(in_features=hidden_dim_4, out_features=n_classes)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
            Args:
                x (torch.Tensor): (batch_size, in_dim) the input

            Output:
                (torch.Tensor): (batch_size, n_classes) the output
        """
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        # x = self.layer5(x)
        x = self.output_layer(x)

        return x

In [48]:
class Data(Dataset):
    def __init__(
        self,
        data
    ):
        n = data.shape[1]
        self.features = torch.tensor(data.iloc[:, 0:n-1].values.astype(np.int64), dtype=torch.float32)
        self.labels = torch.tensor(data.iloc[:, -1].values.astype(np.int64), dtype=torch.int64)

    def __getitem__(self, index):
        return self.features[index], self.labels[index]

    def __len__(self):
        return len(self.features)

In [49]:
X_train, X_test, Y_train, Y_test = train_test_split(X_final, Y, test_size=0.2, random_state=42, stratify=Y)
X_train = pd.DataFrame(X_train)
Y_train = pd.DataFrame(Y_train)
X_test = pd.DataFrame(X_test)
Y_test = pd.DataFrame(Y_test)

X_train = pd.concat([X_train, Y_train], axis=1)
X_test = pd.concat([X_test, Y_test], axis=1)
train_dataset = Data(data=X_train)
test_dataset = Data(data=X_test)

In [50]:
train_batchsize = 512
val_batchsize = 512

train_dataloader = DataLoader(dataset=train_dataset, batch_size=train_batchsize, shuffle=True)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=val_batchsize, shuffle=True)

In [51]:
n_epochs = 5

In [52]:
in_dim = X_final.shape[1]
in_dim

2282

In [53]:
model = ANN(
	in_dim = in_dim,
	hidden_dim_1 = in_dim // 2,
	hidden_dim_2 = in_dim // 4,
	hidden_dim_3 = in_dim // 8,
  hidden_dim_4 = 3
).to(device)

In [54]:
lr = 1e-3
optimiser = torch.optim.Adam(model.parameters(), lr=lr)
loss_fn = torch.nn.CrossEntropyLoss()

In [55]:
sanity_check=False

In [56]:
def train_epoch(
    model,
    dataloader,
    optimiser
):
    model.train()

    for batch in tqdm(dataloader):
        x, y = batch[0], batch[1]

        output = model(x)
        output = nn.Softmax(dim=-1)(output)
        loss = loss_fn(output, y)

        optimiser.zero_grad()
        loss.backward()
        optimiser.step()

        if sanity_check:
            break

def validate(
    model,
    dataloader
):
    model.eval()
    total_loss = 0
    predictions = []
    truths = []

    with torch.no_grad():
        for batch in tqdm(dataloader):
            x, y = batch[0], batch[1]

            output = model(x)
            output = nn.Softmax(dim=-1)(output)
            loss = loss_fn(output, y)
            total_loss += loss.detach().cpu().item()/len(dataloader)

            preds = torch.argmax(output, dim=-1)
            predictions.extend(preds.cpu())
            truths.extend(y.cpu())

            if sanity_check:
                break

    acc = accuracy_score(y_true=truths, y_pred=predictions)
    f1 = f1_score(y_true=truths, y_pred=predictions, average='macro')

    return total_loss, acc, f1

In [57]:
def train_model(
    model,
    train_dataloader,
    test_dataloader,
    optimiser
):
    for epoch in range(1, n_epochs+1):
        start_time = time.time()

        print(f"========= EPOCH {epoch} STARTED =========")
        train_epoch(model=model, dataloader=train_dataloader, optimiser=optimiser)

        print(f"========= TRAIN EVALUATION STARTED =========")
        train_val_op = validate(model=model, dataloader=train_dataloader)

        print(f"========= TEST EVALUATION STARTED =========")
        test_val_op = validate(model=model, dataloader=test_dataloader)

        print(f"END OF {epoch} EPOCH")
        print(f"| Time taken: {time.time() - start_time: 7.3f} |")
        print(f"| Train Loss: {train_val_op[0]: 7.3f} | Train acc: {train_val_op[1]: 1.5f} | Train f1: {train_val_op[2]: 1.5f} |")
        print(f"| Test Loss: {test_val_op[0]: 7.3f}  | Test acc: {test_val_op[1]: 1.5f}  | Test f1: {test_val_op[2]: 1.5f}  |")

        if sanity_check:
            break

In [58]:
train_model(
    model=model,
    train_dataloader=train_dataloader,
    test_dataloader=test_dataloader,
    optimiser=optimiser
)

========= EPOCH 1 STARTED =========


100%|██████████| 112/112 [00:26<00:00,  4.19it/s]


========= TRAIN EVALUATION STARTED =========


100%|██████████| 112/112 [00:13<00:00,  8.13it/s]


========= TEST EVALUATION STARTED =========


...

========= TRAIN EVALUATION STARTED =========


100%|██████████| 112/112 [00:13<00:00,  8.15it/s]


========= TEST EVALUATION STARTED =========


100%|██████████| 28/28 [00:02<00:00, 10.74it/s]


END OF 5 EPOCH
| Time taken:  51.251 |
| Train Loss:   0.833 | Train acc:  0.72678 | Train f1:  0.50034 |
| Test Loss:   0.844  | Test acc:  0.71414  | Test f1:  0.49111  |


In [59]:
test_df_encoded

,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,...,rosiglitazone_Steady,rosiglitazone_Up,insulin_Down,insulin_No,insulin_Steady,insulin_Up,change_Ch,change_No,diabetesMed_No,diabetesMed_Yes
0,5,5,17,12,25,3,29,0,0,4,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
1,2,3,7,3,59,0,24,0,1,0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
2,3,1,1,8,47,0,23,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,1,3,7,10,46,6,38,0,2,7,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
4,1,6,7,5,64,1,16,0,0,1,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30525,1,6,7,2,44,0,19,0,2,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
30526,3,18,4,2,19,3,10,0,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
30527,1,3,7,5,73,3,24,0,0,1,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
30528,1,3,7,2,63,0,13,0,0,1,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0


In [60]:
test_df_encoded = torch.tensor(test_df_encoded.values.astype(np.int64), dtype=torch.float32)
print(test_df_encoded)
# Assuming model is your PyTorch model
predicted_labels = torch.nn.Softmax(dim=-1)(model(test_df_encoded))
predicted_labels = torch.max(predicted_labels.data, 1)

tensor([[ 5.,  5., 17.,  ...,  1.,  0.,  1.],
        [ 2.,  3.,  7.,  ...,  0.,  0.,  1.],
        [ 3.,  1.,  1.,  ...,  0.,  0.,  1.],
        ...,
        [ 1.,  3.,  7.,  ...,  0.,  0.,  1.],
        [ 1.,  3.,  7.,  ...,  0.,  0.,  1.],
        [ 1.,  1.,  7.,  ...,  1.,  0.,  1.]])


In [61]:
predicted_labels = predicted_labels.indices.numpy()

In [62]:
subm = pd.read_csv("/content/sample_submission.csv")
subm["readmission_id"] = predicted_labels
subm.to_csv('submission.csv', index=False)